In [5]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, PatchTST

from neuralforecast.losses.pytorch import MAE
from neuralforecast.losses.numpy import mae, mse

from datasetsforecast.long_horizon import LongHorizon


In [6]:
df = pd.read_csv("./datasets/DCFC_load.csv")
load_df = df.drop((['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7']),axis=1)
time = pd.to_datetime(load_df['Time'], format='%d/%m/%y %H:%M')

time_train = time[:23328]
x_train = load_df["PowerkW2"][:23328]

time_cv = time[23328:]
x_cv = load_df["PowerkW2"][23328:]

In [7]:
import pandas as pd

# Assuming your existing DataFrame is named load_df
# If not, replace 'load_df' with the actual name of your DataFrame
load_df['Time'] = pd.to_datetime(load_df['Time'], format='%d/%m/%y %H:%M')

# Create a new DataFrame with 15-minute intervals
start_time = load_df['Time'].min()
end_time = load_df['Time'].max()
new_time_range = pd.date_range(start=start_time, end=end_time, freq='15T')
new_df = pd.DataFrame({'Time': new_time_range})

# Replace the 'Time' column in the original DataFrame with the new one
load_df['Time'] = new_df['Time']

new_row = {'Time': pd.Timestamp('2022-01-02 00:00:00'), 'PowerkW2': 0.0}

# Find the index of the row with the specified timestamp
index_of_timestamp = load_df[load_df['Time'] == pd.Timestamp('2022-01-01 23:45:00')].index[0]

# Insert the new row after the specified timestamp
load_df = pd.concat([load_df.iloc[:index_of_timestamp + 1], pd.DataFrame([new_row], columns=load_df.columns), load_df.iloc[index_of_timestamp + 1:]], ignore_index=True)

load_df.drop(96, inplace=True)

In [8]:
load_df['Time'][35040] = pd.Timestamp('2022-12-31 23:45:00')
load_df[135:150]

,Time,PowerkW2
136,2022-01-02 09:45:00,0.000000
137,2022-01-02 10:00:00,0.000000
138,2022-01-02 10:15:00,0.000000
139,2022-01-02 10:30:00,0.000000
140,2022-01-02 10:45:00,0.000000
141,2022-01-02 11:00:00,0.000000
142,2022-01-02 11:15:00,0.000000
143,2022-01-02 11:30:00,0.000000
144,2022-01-02 11:45:00,0.000000
145,2022-01-02 12:00:00,0.000000


In [9]:
load_df["PowerkW2"] = load_df["PowerkW2"].astype(np.float32)

In [19]:
load_df['ds'] = pd.to_datetime(load_df['Time'])

In [21]:
load_df.drop('Time', axis=1, inplace=True)

In [22]:
load_df['unique_id'] = 0

In [24]:
load_df.rename(columns={'PowerkW2': 'y'}, inplace=True)

In [25]:
val_size = 11712
test_size = 23328

In [26]:
horizon = 672

models = [NHITS(h=horizon,
               input_size=2*horizon,
               max_steps=10),
         NBEATS(h=horizon,
               input_size=2*horizon,
               max_steps=10),
         PatchTST(h=horizon,
                 input_size=2*horizon,
                 max_steps=10)]

Seed set to 1
Seed set to 1
Seed set to 1


In [27]:
nf = NeuralForecast(models=models, freq='D')

In [28]:
preds_df = nf.cross_validation(df=load_df, val_size=val_size, test_size=test_size, n_windows=None)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

NotImplementedError: The operator 'aten::upsample_linear1d.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
y_true = preds_df['y'].values
y_pred_nhits = preds_df['NHITS'].values
y_pred_nbeats = preds_df['NBEATS'].values
y_pred_patchtst = preds_df['PatchTST'].values

n_series = len(Y_df['unique_id'].unique())

y_true = y_true.reshape(n_series, -1, horizon)
y_pred_nhits = y_pred_nhits.reshape(n_series, -1, horizon)
y_pred_nbeats = y_pred_nbeats.reshape(n_series, -1, horizon)
y_pred_patchtst = y_pred_patchtst.reshape(n_series, -1, horizon)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(y_true[0, 0, :], label='True')
ax.plot(y_pred_nhits[0, 0, :], label='N-HiTS', ls='--')
ax.plot(y_pred_nbeats[0, 0, :], label='N-BEATS', ls=':')
ax.plot(y_pred_patchtst[0, 0, :], label='PatchTST', ls='-.')
ax.set_ylabel('Exchange rate')
ax.set_xlabel('Forecast horizon')
ax.legend(loc='best')

plt.tight_layout()

In [ ]:
data = {'N-HiTS': [mae(y_pred_nhits, y_true), mse(y_pred_nhits, y_true)],
       'N-BEATS': [mae(y_pred_nbeats, y_true), mse(y_pred_nbeats, y_true)],
       'PatchTST': [mae(y_pred_patchtst, y_true), mse(y_pred_patchtst, y_true)]}

metrics_df = pd.DataFrame(data=data)
metrics_df.index = ['mae', 'mse']

metrics_df.style.highlight_min(color='lightgreen', axis=1)